In this notebook, we are going to see how a constraint can be activated under a variable-based condition (ie, an other constraint).
The small problem we are going to solve is the following:

%%latex
\begin{equation*}
  f(a,b)=\begin{cases}
    a\times b, & \text{if $a>b$}.\\
    a+b, & \text{otherwise}.
  \end{cases}
\end{equation*}

There are three ways to handle such conditional constraint in Choco.

In [1]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-snapshots</id>
  <url>https://oss.sonatype.org/content/repositories/snapshots/</url>
</repository>
<dependency>
  <groupId>org.choco-solver</groupId>
  <artifactId>choco-solver</artifactId>
  <version>4.10.0</version>
</dependency>

In [2]:
import org.chocosolver.solver.Model;
import org.chocosolver.solver.variables.IntVar;
import org.chocosolver.solver.variables.BoolVar;
import org.chocosolver.solver.Solver;

## First approach
In the first approach, we are going to use a `ifThenElse` constraint that takes three constraints as parameters.
The first constraint is the conditional, the second is the one that is satisfied when the condition holds, 
the third is the one that is satified when the condition doesn't hold. 

In [3]:
{
    Model model = new Model("1st try");
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);
    
    model.ifThenElse(
        model.arithm(A,">",B), // if A > B
        model.times(A, B, F), // then F = A * B
        model.arithm(A,"+",B,"=",F)); // else F = A + B

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, H=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

Model[1st try], 9 variables, 5 constraints, building time: 0,054s, w/o user-defined search strategy, w/o complementary search strategy
A=0, B=1, H=1
A=0, B=2, H=2
A=0, B=3, H=3
A=1, B=1, H=2
A=1, B=2, H=3
A=1, B=3, H=4
A=2, B=3, H=5
A=3, B=3, H=6
A=0, B=0, H=0
A=2, B=2, H=4
A=1, B=0, H=0
A=2, B=0, H=0
A=3, B=0, H=0
A=2, B=1, H=2
A=3, B=1, H=3
A=3, B=2, H=6
Model[1st try], 16 Solutions, Resolution time 0,055s, 35 Nodes (639,2 n/s), 39 Backtracks, 0 Backjumps, 4 Fails, 0 Restarts


In that approach, some additional variables and constraints are introduced to make sure the relation holds.
It relies on reification which attaches a boolean variable to a constraint, which is set to `true` when the constraint is satified, 
`false` otherwise (and vice versa). In this approach, the decomposition is automatically done by the model on the call to `ifThenElse`.

## Second approach: using expressions
Variables can be combined together to form an expression. An expression can be arithmetical, logical or relational.
Once declared, any expression can be turned into an integer variable (for arithmetical expression), a boolean variable (for logical expression) or a constraint (for relationnal one). 
The latter can either be posted as a combination of constraints or as a table constraint. In that case, the tuple are generated from the expression (this may not be "free" in term of time and space).

In [4]:
{
    Model model = new Model("1st try");
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);
    
    F.eq( // F is equal to ...
        A.gt(B).ift( // if A > B 
            A.mul(B), // then A * B
            A.add(B)  // else A + B
        )
    ).decompose() // decompose the expression
    //).extension() // generate a table constraint
    .post(); // post the constraint

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, F=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

Model[1st try], 8 variables, 6 constraints, building time: 0,012s, w/o user-defined search strategy, w/o complementary search strategy
A=1, B=0, F=0
A=2, B=0, F=0
A=3, B=0, F=0
A=2, B=1, F=2
A=3, B=1, F=3
A=3, B=2, F=6
A=0, B=1, F=1
A=1, B=1, F=2
A=0, B=2, F=2
A=1, B=2, F=3
A=0, B=3, F=3
A=1, B=3, F=4
A=2, B=3, F=5
A=3, B=3, F=6
Model[1st try], 14 Solutions, Resolution time 0,023s, 29 Nodes (1 277,3 n/s), 31 Backtracks, 0 Backjumps, 2 Fails, 0 Restarts


In this approach again, the model automatically decomposes the relation into constraints and introduces some variables. One may replace `.decompose()` by `.extension()` and note that no constraint or variable is added. 

## Third approach: DIY
In that approach, constraints are reified with boolean variables and those variables are, in turn, linked to an `if then` relation.

In [5]:
{
    Model model = new Model();
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);

    BoolVar b = model.arithm(A, ">", B).reify();
    model.times(A, B, F).reifyWith(b);
    model.arithm(A, "+", B, "=", F).reifyWith(b.not());

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, H=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

Model[Model-0], 5 variables, 3 constraints, building time: 0,001s, w/o user-defined search strategy, w/o complementary search strategy
A=1, B=0, H=0
A=2, B=0, H=0
A=3, B=0, H=0
A=2, B=1, H=2
A=3, B=1, H=3
A=3, B=2, H=6
A=0, B=1, H=1
A=0, B=2, H=2
A=0, B=3, H=3
A=1, B=1, H=2
A=1, B=2, H=3
A=1, B=3, H=4
A=2, B=3, H=5
A=3, B=3, H=6
Model[Model-0], 14 Solutions, Resolution time 0,034s, 29 Nodes (845,5 n/s), 31 Backtracks, 0 Backjumps, 2 Fails, 0 Restarts


The `A>B` constraint is reified with `b` a boolean variable which is then used to reify the second constraint.
Then, the refutation of `b` is used to reified the third constraint. 

When a constraint is reified with a boolean variable, it should not posted (that would forces the boolean variable to always  be equal to 1). But, the boolean variable can be used to reified other constraints and/or be linked with other variables in other constraints, like `sum` or clauses.